此腳本將存有邊界框的csv檔轉為labelme格式的json檔

In [2]:
from PIL import Image
from glob import glob
import base64
import json
import csv
import io
import os

## 功能定義

In [14]:
def img_data_to_b64(img_pil, format="JPEG"):
    f = io.BytesIO()
    img_pil.save(f, format)
    img_bin = f.getvalue()
    if hasattr(base64, "encodebytes"):
        img_b64 = base64.encodebytes(img_bin)
    else:
        img_b64 = base64.encodestring(img_bin)
    return img_b64


def create_dir(dir_path):
    if dir_path != '':
        if dir_path[-1] == '/':
            dir_path += '/'
        try:
            os.mkdir(dir_path)
        except:
            pass
    
    
def anno_to_labelme(anno_csv, img_dir, out_dir):
    create_dir(out_dir)

    # read bbox csv file
    anno_list = []
    with open(anno_csv) as csvfile:
        rows = csv.reader(csvfile)
        next(rows)
        for row in rows:
            anno_list.append(row)

    # create image list
    shape_dic = dict()
    for anno in anno_list:
        img_name = anno[0]
        shape_dic[img_name] = []

    # save shapes into shape dic
    for bbox in anno_list:
        # create shape template
        shape = dict(
            label=None,
            points=None,
            group_id=None,
            shape_type='rectangle',
            flags={}
            )

        img_name = bbox[0]
        float_point = [float(n) for n in bbox[1:5]]
        points = [float_point[0:2], float_point[2:4]]
        # save point and label to shape
        shape['label'] = bbox[5]
        shape['points'] = points
        # save shape to dic
        shape_dic[img_name].append(shape)

    # save labelme file
    for img_name in shape_dic.keys():
        # read and encode image
        img_path = img_dir + img_name  # path
        img = Image.open(img_path)  # open with PIL
        img_b64 = img_data_to_b64(img)  # encode
        img_b64_utf8 = img_b64.decode('utf-8')  # decode to utf8

        # create data container
        data = dict(
                    version='4.5.7',
                    flags={},
                    shapes=shape_dic[img_name],
                    imagePath=img_name,
                    imageData=img_b64_utf8,
                    imageHeight=img.size[1],
                    imageWidth=img.size[0],
                )

        # save as json file
        json_path = out_dir + os.path.splitext(img_name)[0] + '.json'
        with open(json_path, 'w') as jsonfile:
            json.dump(data, jsonfile, ensure_ascii=False, indent=2)

In [9]:
glob('../../peter/data_temp/*')

['../../peter/data_temp/bbt_sample_class_5',
 '../../peter/data_temp/bbt_sample_class_2',
 '../../peter/data_temp/bbt_sample_class_3']

## 單檔案輸出測試

In [15]:
anno_csv = glob('../../peter/data_temp/bbt_sample_class_5/*.csv')[6]
img_dir = '../../peter/data_temp/bbt_sample_class_5/'
out_dir = 'labelme_test'

anno_to_labelme(anno_csv, img_dir, out_dir)

## 多檔案輸出

In [17]:
anno_csv_list = glob('../../peter/data_temp/bbt_sample_class_5/*.csv')
img_dir = '../../peter/data_temp/bbt_sample_class_5/'
out_dir = '../../peter/data_temp/bbt_sample_class_5/labelme/'


total = len(anno_csv_list)
prgrs = 0
for anno_csv in anno_csv_list:
    anno_to_labelme(anno_csv, img_dir, out_dir)
    prgrs += 1
    print('\r[Progress]:[{:<30s}] {:5.1f}%'.format('█'*int(prgrs*30/total), 
                                                   prgrs/total*100), end='')
    
print('\nDone')

[Progress]:[██████████████████████████████] 100.0%
Done
